## Micromobility MNL Model 0-5 Miles

In [1]:
# Importing Packages and Data
import pandas as pd
import numpy as np
import biogeme as biogeme
import biogeme.database as db
import biogeme.biogeme as bio
from biogeme import models
import biogeme.messaging as msg
from biogeme.expressions import Beta
from biogeme.expressions import (
    Beta,
    DefineVariable,
    bioDraws,
    PanelLikelihoodTrajectory,
    MonteCarlo,
    log,
    Derive
)
import math
from datetime import datetime
import matplotlib.pyplot as plt

In [2]:
# import data
micro_pool_socio = pd.read_csv('Data/micro_pool_socio_bio2up.csv')

micro_pool_socio['PRCP_Yes'] = np.where((micro_pool_socio['PRCP'] == 1) | (micro_pool_socio['PRCP'] == 2), 1, 0)
micro_pool_socio['BKLN_Yes'] = np.where((micro_pool_socio['BKLN'] == 1) | (micro_pool_socio['BKLN'] == 2) | (micro_pool_socio['BKLN'] == 3), 1, 0)

In [3]:
# Micromobility Trips 0-5 Miles
micro0to5 = (((
# 0-2 miles first choice
micro_pool_socio['scooter_av'] == 1) & (
    micro_pool_socio['dlbike_av'] == 1) & (
    micro_pool_socio['dkbike_av'] == 1) & (
    micro_pool_socio['car_av'] == 0) & (
    micro_pool_socio['transit_av'] == 0) & (
    micro_pool_socio['rd_av'] == 0) & (
    micro_pool_socio['walk_av'] == 0) & (
    micro_pool_socio['bike_av'] == 0) & (
    micro_pool_socio['sctransit_av'] == 0)) | ((
# 0-2 miles second choice
    # scooter_av == 1 & dlbike_av == 1 & everything else == 0
    micro_pool_socio['scooter_av'] == 1) & (
    micro_pool_socio['dlbike_av'] == 1) & (
    micro_pool_socio['dkbike_av'] == 0) & (
    micro_pool_socio['car_av'] == 0) & (
    micro_pool_socio['transit_av'] == 0) & (
    micro_pool_socio['rd_av'] == 0) & (
    micro_pool_socio['walk_av'] == 0) & (
    micro_pool_socio['bike_av'] == 0) & (
    micro_pool_socio['sctransit_av'] == 0)) | ((
    #'scooter'_av ==1 & 'dk_bike'_av == 1 & everything else == 0
    micro_pool_socio['scooter_av'] == 1) & (
    micro_pool_socio['dkbike_av'] == 1) & (
    micro_pool_socio['dlbike_av'] == 0) & (
    micro_pool_socio['car_av'] == 0) & (
    micro_pool_socio['transit_av'] == 0) & (
    micro_pool_socio['rd_av'] == 0) & (
    micro_pool_socio['walk_av'] == 0) & (
    micro_pool_socio['bike_av'] == 0) & (
    micro_pool_socio['sctransit_av'] == 0)) | ((
    #'dk_bike_av' == 1 & 'dl_bike_av' == 1 & everything else == 0
    micro_pool_socio['dkbike_av'] == 1) & (
    micro_pool_socio['dlbike_av'] == 1) & (
    micro_pool_socio['scooter_av'] == 0) & (
    micro_pool_socio['car_av'] == 0) & (
    micro_pool_socio['transit_av'] == 0) & (
    micro_pool_socio['rd_av'] == 0) & (
    micro_pool_socio['walk_av'] == 0) & (
    micro_pool_socio['bike_av'] == 0) & (
    micro_pool_socio['sctransit_av'] == 0
    )) | ((
# 2-5 miles
    micro_pool_socio['scooter_av'] == 1) & (
    micro_pool_socio['dlbike_av'] == 0) & (
    micro_pool_socio['dkbike_av'] == 0) & (
    micro_pool_socio['car_av'] == 0) & (
    micro_pool_socio['transit_av'] == 0) & (
    micro_pool_socio['rd_av'] == 0) & (
    micro_pool_socio['walk_av'] == 0) & (
    micro_pool_socio['bike_av'] == 0) & (
    micro_pool_socio['sctransit_av'] == 1
)))

micro_0_to_5 = micro_pool_socio.loc[micro0to5]

In [4]:
database4 = db.Database('micro_0_to_5', micro_0_to_5)
globals().update(database4.variables)

In [8]:
ASC_SCOOTER = Beta('ASC_SCOOTER', 0, None, None, 0)
ASC_SCOOTER_PRIME = Beta('ASC_SCOOTER_PRIME', 0, None, None, 0)
ASC_DLBIKE = Beta('ASC_DLBIKE', 0, None, None, 0)
ASC_DKBIKE = Beta('ASC_DKBIKE', 0, None, None, 0)
ASC_SCTRANSIT = Beta('ASC_SCTRANSIT', 0, None, None, 1)

B_SCOOTERTIME = Beta('B_SCOOTERTIME', 0, None, None, 0)
B_SCOOTERTIME_PRIME = Beta('B_SCOOTERTIME_PRIME', 0, None, None, 0)
B_BIKETIME = Beta('B_BIKETIME', 0, None, None, 0)
# B_DKBIKETIME = Beta('B_DKBIKETIME', 0, None, None, 0)
B_SCTRANSITTIME = Beta('B_SCTRANSITTIME', 0, None, None, 0)

B_ACCESS = Beta('B_ACCESS', 0, None, None, 0)
B_DROP =Beta('B_DROP', 0, None, None, 0)
B_WAITAV = Beta('B_WAITAV', 0, None, None, 0)
B_AV = Beta('B_AV', 0, None, None, 0)

B_COST = Beta('B_COST', 0, None, None, 0)
B_COST_PRIME = Beta('B_COST_PRIME', 0, None, None, 0)
B_costadj = Beta('B_costadj', 0, None, None, 0)

B_PRCP_Yes = Beta('B_PRCP_Yes', 0, None, None, 0)
B_OWNBIKE = Beta('B_OWNBIKE', 0, None, None, 0)
B_BIKELANE = Beta('B_BIKELANE', 0, None, None, 0)
B_AGE_SQ = Beta('B_AGE_SQ', 0, None, None, 0)
B_AGE_SCTRANSIT_SQ = Beta('B_AGE_SCTRANSIT_SQ', 0, None, None, 0)
B_GENDER_F = Beta('B_GENDER_F', 0, None, None, 0)

In [12]:
# Utility Functions
V7 = (ASC_SCOOTER   
      + ASC_SCOOTER_PRIME*AVtech
      + B_SCOOTERTIME*sctime
      + B_SCOOTERTIME_PRIME*sctime*AVtech
      + B_COST*sccost_adj
      + B_COST_PRIME*sccost_adj*AVtech
      + B_ACCESS*SCAW*(1-AVtech)
      + B_WAITAV*SCAV*AVtech 
      )
V8 = (ASC_DLBIKE
        + B_BIKETIME*dltime
        + B_COST*dlcost_adj
        + B_ACCESS*DLAW
        # + B_PRCP_Yes*PRCP_Yes
        # + B_OWNBIKE*bike
        # + B_BIKELANE*BKLN_Yes
        # + B_AGE_SQ*age*age
        # + B_GENDER_F*gender_F
    )
V9 = (ASC_DKBIKE 
      + B_BIKETIME*dbtime 
      + B_COST*dbcost_adj
      + B_ACCESS*DBAW
      # + B_DROP*DBDW
      # + B_OWNBIKE*bike
      # + B_BIKELANE*BKLN_Yes
      # + B_AGE_SQ*age*age
      # + B_GENDER_F*gender_F
    )
V10 = (ASC_SCTRANSIT
       + B_SCTRANSITTIME*sttotime
       + B_COST*sttocost_adj 
       + B_ACCESS*STAW*(1-AVtech)
       + B_WAITAV*STAV*AVtech 
      #  + B_AGE_SCTRANSIT_SQ*age*age
    )

In [13]:
V = {7: V7, 8: V8, 9: V9, 10: V10} # Associating the utility functions with the numbering of the alternatives
av = {7: scooter_av, 8: dlbike_av, 9: dkbike_av, 10: sctransit_av} # Associating the availability conditions with the alternatives

logprob = models.loglogit(V, av, choice)
biogeme = bio.BIOGEME(database4, logprob)
biogeme.modelName = 'micromobility_mnl_0_to_5'

biogeme.calculateNullLoglikelihood(av)
model_results = biogeme.estimate()
pandasResults = model_results.getEstimatedParameters()
print(pandasResults.round(2))

# get AIC value of model_results
# AIC = model_results.data.AIC

                     Value  Rob. Std err  Rob. t-test  Rob. p-value
ASC_DKBIKE            0.27          0.24         1.13          0.26
ASC_DLBIKE            1.05          0.24         4.40          0.00
ASC_SCOOTER           0.80          0.22         3.59          0.00
ASC_SCOOTER_PRIME     0.34          0.16         2.12          0.03
B_ACCESS             -0.11          0.01        -7.40          0.00
B_BIKETIME           -0.03          0.02        -1.67          0.10
B_COST               -1.82          0.30        -6.17          0.00
B_COST_PRIME          0.28          0.26         1.06          0.29
B_SCOOTERTIME        -0.05          0.01        -3.57          0.00
B_SCOOTERTIME_PRIME  -0.01          0.01        -1.35          0.18
B_SCTRANSITTIME       0.00          0.02         0.19          0.85
B_WAITAV             -0.03          0.02        -1.33          0.18


In [ ]:
print(logLike)